# Quantum Models - Training & Evaluation

This notebook trains **Quantum Baselines** for the comparative study.

**Models Trained:**
1. Quantum Kernel SVM (QSVM)
2. Variational Quantum Neural Network (VQNN)

**Constraint:** Quantum simulation is slow, so we subsample the training data.
**Output:** `results/tables/quantum_results.csv`

In [1]:
import pandas as pd
import numpy as np
import time
import os
import sys

# Ensure project root is in path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.data.load_data import load_config, load_creditcard_data
from src.data.preprocess import preprocess_creditcard
from src.features.dimensionality_reduction import apply_pca
from src.utils.seed import set_seed

# Quantum Models
from src.quantum.quantum_kernel import QuantumKernelModel
from src.quantum.variational_qnn import VQNNTrainer

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configuration
config_path = os.path.join(project_root, 'src', 'config', 'config.yaml')
config = load_config(config_path)
N_QUBITS = config['quantum']['n_qubits']
SEED = 42
set_seed(SEED)

print(f"Using Feature Dimension (PCA): {N_QUBITS}")

Random seed set to 42
Using Feature Dimension (PCA): 8


In [3]:
# Load & Process Data
df = load_creditcard_data(config)
df_clean = preprocess_creditcard(df, config)

X_all = df_clean.drop(columns=['Class']).values
y_all = df_clean['Class'].values

# Apply PCA
X_pca, _ = apply_pca(X_all, n_components=N_QUBITS)

# Split & Subsample for Quantum
from sklearn.model_selection import train_test_split

# Subsample: 1000 Train, 500 Test (for feasible simulation time)
X_sample, _, y_sample, _ = train_test_split(X_pca, y_all, train_size=1500, stratify=y_all, random_state=SEED)
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.33, stratify=y_sample, random_state=SEED)

print(f"Quantum Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

2026-01-28 13:30:21,548 INFO Loading Credit Card data from c:\Users\mostr\OneDrive\Documents\GitHub\Anomaly-detection-in-finance-using-QML\data/raw\creditcard.csv
2026-01-28 13:30:25,309 INFO Loaded Credit Card data with shape (284807, 31)
2026-01-28 13:30:26,908 INFO Cleaned data: (284807, 31) -> (283726, 31)
2026-01-28 13:30:27,153 INFO Normalized 29 columns using minmax
2026-01-28 13:30:27,411 INFO PCA reduced to 8 components. Explained Variance: 1.0000
Quantum Train Shape: (1005, 8), Test Shape: (495, 8)


In [ ]:
def train_evaluate_quantum(model, name, X_train, y_train, X_test, y_test):
    print(f"Training {name}...")
    start = time.time()
    
    model.fit(X_train, y_train)
        
    train_time = time.time() - start
    
    # Predict
    start_inf = time.time()
    # VQNN might return binary, try to get probability if possible or just use predict
    scores = model.predict(X_test)
    inf_time = time.time() - start_inf
    
    from sklearn.metrics import average_precision_score, roc_auc_score , accuracy
    rc = roc_auc_score(y_test, scores)
    pr = average_precision_score(y_test, scores)
    acc = accuracy(y_test, scores)
    
    return {
        "model": name,
        "type": "quantum",
        "roc_auc": rc,
        "pr_auc": pr,
        "accuracy" : acc,
        "train_time": train_time,
        "inference_time": inf_time
    }

In [5]:
models = [
    ("QSVM", QuantumKernelModel(n_qubits=N_QUBITS)),
    ("VQNN", VQNNTrainer(n_qubits=N_QUBITS, n_layers=2, epochs=10))
]

results = []
for name, model in models:
    try:
        res = train_evaluate_quantum(model, name, X_train, y_train, X_test, y_test)
        results.append(res)
        print(f"   -> PR-AUC: {res['pr_auc']:.4f}")
    except Exception as e:
        print(f"   Failed {name}: {e}")

Training QSVM...
2026-01-28 13:30:31,442 WARNING Training set too large for Quantum Kernel. Subsampling to 500.
2026-01-28 13:30:31,444 INFO Computing Quantum Kernel Matrix: 500x500
2026-01-28 13:55:46,458 INFO Fitting SVC with Quantum Kernel...
2026-01-28 13:55:46,657 INFO Computing Quantum Kernel Matrix: 495x500
   -> PR-AUC: 0.0020
Training VQNN...
2026-01-28 14:20:51,374 INFO Training VQNN...
2026-01-28 14:21:04,239 INFO Epoch 5, Loss: 0.0196
2026-01-28 14:21:10,018 INFO Epoch 10, Loss: 0.0142
   -> PR-AUC: 0.0020


In [6]:
# Save Results
df_res = pd.DataFrame(results)
os.makedirs(os.path.join(project_root, 'results', 'tables'), exist_ok=True)
df_res.to_csv(os.path.join(project_root, 'results', 'tables', 'quantum_results.csv'), index=False)
df_res

,model,type,roc_auc,pr_auc,train_time,inference_time
0,QSVM,quantum,0.5,0.00202,1515.205777,1504.588445
1,VQNN,quantum,0.5,0.00202,18.720392,0.065963
